##### import libraries 

In [1]:
#pip install scikit-learn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

##### load the data into pandas data frame

In [3]:
train_df = pd.read_csv('data/train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##### drop columns with minimal importance(PassengerId, Ticket)

In [4]:
#rain_df.drop(columns = ['Ticket','PassengerId'], axis=1, inplace=True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
train_df.info() ###check to see which columns have missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


##### create new classes for columns that may have additional info that may be used later 

In [6]:
def assignDeckValue(CabinCode):
    if pd.isnull(CabinCode):
        category = 'Unknown'
        
    else:
        
        category = CabinCode[0]
        
    return category


Deck = np.array([assignDeckValue(cabin) for cabin in train_df['Cabin'].values])

train_df = train_df.assign(Deck = Deck)

train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1



##### extract title from the Name of Passenger 

In [7]:
train_df['Title'] = train_df.Name.str.extract(' ([A-Za-z+])\.',expand=False)


train_df['Title'] = train_df['Title'].replace(['Dr','Rev','Col','Major','Countess','Sir','Jonkheer','Lady','Capt','Don'],'Others')
train_df['Title'] = train_df['Title'].replace('Ms','Miss')
train_df['Title'] = train_df['Title'].replace('Mme','Mrs')
train_df['Title'] = train_df['Title'].replace('Mlle','Miss')


In [8]:
train_df.drop(columns = ['Cabin','Name'],axis=1,inplace=True)

In [9]:
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck,FamilySize,Title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,S,Unknown,2,NaN
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C,C,2,NaN
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,Unknown,1,NaN
3,4,1,1,female,35.0,1,0,113803,53.1000,S,C,2,NaN
4,5,0,3,male,35.0,0,0,373450,8.0500,S,Unknown,1,NaN


In [10]:

        
train_df['Embarked'].value_counts()

common_occurence = 'S'

train_df['Embarked']=train_df['Embarked'].fillna('S')

train_df['Embarked'].isnull().sum()

0

In [ ]:
#### impute the missing values with the most commonly occuring value in the column

means = train_df.groupby('Title')['Age'].mean()

title_list = ['Master','Miss','Mr','Mrs','Others']
def age_missing_replace(means,dframe,title_list):
    for title in title_list:
        temp = dframe['Title'] == title
        dframe.loc[temp,'Age'] = dframe.loc[temp, 'Age'].fillna(means[title])
        

age_missing_replace(means,train_df, title_list)


In [ ]:
####map the categorical values into numerical data using map()

train_df['Embarked'] = train_df['Embarked'].map({'C':0,'Q':1,'S':2})
train_df['Sex'] = train_df['Sex'].map({'male':0,'female':1})
train_df['Title'] = train_df['Title'].map({'Master':0,'Miss':1,'Mr':2,'Mrs':3,'Others':4})


#### transform train_df into numerical value using sklearn.preprocessing library

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_df['Deck'] = le.fit_transform(train_df['Deck'])

In [ ]:
train_df.head()

In [ ]:
train_df.info()